# White Noise: Data Collection

## 1. Explaining the Problem

To start, I visually inspect the api.congress.gov website. I establish that my main API endpoint of interest is the `bill` endpoint, which can be easily paginated by filtering by US Congress sessions - i.e., `{congress}` - bill types - i.e., `{billType}`, taking the value `hr` if the bill was introduced in the House of Representatives, or the value `s` if the bill was introduced in the Senate - and, when needed, bill number - i.e., `{billNumber}`, which helps to return detailed information for the specified bill. Other `{billType}` values, such as `hjres`, `sjres`, `hconres`, `sconres`, `hres`, or `sres`, refer to various types of US Congress resolutions, which are not of substantive interest because these are by nature bipartisan, thus rarely polarising, or concerning the Republicans' or Democrats' specific political priorities. My theory is focused on the over-time changes of the legislative production by the Republicans in the House of Representatives, and the Senate, during the start of the Obama and Trump administrations - i.e., 2008-10 and 2016-18. Consequently, I refer to the 111th and the 115th US Congress mandates.

Due to how the API endpoints are structured, I must first get all the bill numbers for each Congressional mandate and chamber, keeping them separate and identifiable because these might overlap. In other words, the bill numbers may become indistinguishable if I were to collect them all in a single item, or store them all under a single dictionary key. Only subsequently I can parse over all bill numbers for all Conngressional mandates and chambers, and get each bill's textual summary from the `/bill/{congress}/{billType}/{billNumber}/summaries` API endpoint. These summaries will constitute my main features of interest, since they will allow me to automatically classify each bill - i.e., legislative proposal - by its Economic / Non-Economic, or Socio-Cultural / Non-Socio-Cultural, content. Lastly, I will get the bill metadata from the `bill/{congress}/{billType}/{billNumber}` API endpoint, which I will employ for my data analysis.

In [1]:
# Packages for handling APIs and .JSON files
import os
import requests
import json

# Packages for exercising the virtue of patience and monitoring loop completion
import time
from IPython.display import clear_output

## 2. Getting all the Bill Numbers

I initiate the process of getting all the bill numbers by saving my API key as an environment variable. This helps me in keeping the API key secret, since I cannot include it in the notebook's final version for privacy reasons. Moreover, I specify the API endpoints' base URL - i.e., `api.congress.gov`, and I utilise the `time` package to set appropriate time delays in seconds between the API requests. I do this because the `congress.gov` API only allows for 1000 requests per hour - i.e., one request per 3.6 seconds.

In [2]:
# Setting the API key as an environment variable
os.environ["api_key"] = "I used to be an adventurer like you until I took an arrow to the knee."

# Setting the base URL
base_url = 'https://api.congress.gov'

# Setting a delay of 4 seconds between API requests
rate_limit_delay = 4

I now define function to get all the bill numbers. This takes the US Congress number - i.e., `congress_number` - the bill type - i.e., `bill_type`, which indicates the chamber where the bill was introduced - the file type - i.e., `file_type`, to get a `.JSON` response - the starting number for pagination - i.e., `start_offset` - and the max limit of bill numbers for each page - i.e., `max_limit` - as its arguments. I then call a function within a loop that makes it so the function is utilised to get two separate dictionaries of bill numbers - i.e., `bill_numbers_house`, and `bill_numbers_senate` - with the US Congress numbers as the main keys for the respective dictionaries. Under these keys, the bill numbers are saved as values in a list format.

In [3]:
def get_bill_numbers(congress_number, bill_type, file_type, start_offset, max_limit):
    
    # I define the starting endpoint, which employs the function's arguments instead of hard-coding values.
    endpoint = f"https://api.congress.gov/v3/bill/{congress_number}/{bill_type}?&format={file_type}&offset={start_offset}&limit={max_limit}&api_key={os.environ.get('api_key')}"
    
    # I create an empty list to temporarily store bill numbers at each iteration of the following loop.
    bill_numbers = []
    
    # I open a loop that goes on until there is a KeyError message.
    # Pagination ends when there is no "next" sub-key under the "pagination" key in the .JSON endpoint response.
    
    while True:
        try:
            response = requests.get(endpoint)
            
            # I design a new "bill_type_message" string variable, which helps me in pretty printing a message for each loop
            # iteration, to let myself (and the eventual researcher who wishes to replicate this code) get a tentative idea
            # regarding the loop's status and completion.
            
            if bill_type == "hr":
                bill_type_message = "House of Representatives"
                
            elif bill_type == "s":
                bill_type_message = "Senate"
            
            # If for some reason the "bill_type" variable is not correctly defined, I print an error message and break the loop.
            else:
                print(f"Error retrieving data for the {congress_number}th {bill_type_message}, at page {page}")
                time.sleep(rate_limit_delay) # I still include the delay, because this is still technically an API request.
                break
            
            # In the API documentation at https://api.congress.gov, it is stated that if the response's status code is not 200,
            # then there was an error in retrieving data. I thus devise an appropriate check, which breaks the loop when the
            # status code is not 200, and makes it so that the computer re-tries to get data from that same endpoint.
            
            # I do not make the whole loop stop because I believe it is much more common to encounter temporary errors than
            # errors for which I should just restart the procedure.
            
            if response.status_code != 200:
                print(f"Error retrieving data for the {congress_number}th {bill_type_message}, at page {page}")
                time.sleep(rate_limit_delay) # I still include the delay, because this is still technically an API request.
                break
            
            # I print a message that keeps the user updated on the pagination script's status
            print(f"Getting bill numbers for the {congress_number}th {bill_type_message}, at page {start_offset//max_limit}...")
            
            # I increase the pagination script's starting value to get the right page number at the next iteration
            start_offset += max_limit
            
            # I save the .JSON response into a local variable - i.e., a dictionary
            congress_data = response.json()

            # I get the loop over the data that is saved under the "bills" key in the .JSON response...
            for output in congress_data["bills"]:
                number = output["number"] # I only want the bill number, saved as a value under the "number" key
                
                # I include a check to only append unique bill numbers to the "bill_numbers" list
                if number not in bill_numbers:
                    bill_numbers.append(number)
            
            # I re-define the API endpoint, which is conveniently placed under the "next" sub-key under the "pagination" key
            # in the .JSON API response.
            endpoint = congress_data["pagination"]["next"] + f"&api_key={os.environ.get('api_key')}"

            # I instruct the computer to take a 4 seconds break to avoid overshooting the limit of hourly requests.
            time.sleep(rate_limit_delay)

        # If the computer encounters a KeyError, it is because the "next" sub-key does not exist, thus the program has come up
        # to the very last page. I break the loop because there is no more data to collect.
        except KeyError:
            print(f"The last page for the {congress_number}th {bill_type_message} has been reached.\n")
            break
    
    # The function returns a dictionary with all bill numbers from either the US House of Representative, or Senate, with the
    # US Congress numbers as keys, and the bill numbers as values.
    return bill_numbers

# I am substantively interested in the 111th and the 115th US Congress mandates.
congress_numbers = [111, 115]

# I create empty dictionaries where I can store the bill numbers...
bill_numbers_house = {} # ...for the House of Representatives.
bill_numbers_senate = {} # ...and for the Senate.

# I now call the "get_bill_numbers" function within this loop, to get all the bill numbers into the two dictionaries.
for congress in congress_numbers:
    bill_numbers_house[congress] = get_bill_numbers(congress, "hr", "json", 0, 250)
    bill_numbers_senate[congress] = get_bill_numbers(congress, "s", "json", 0, 250)

Getting bill numbers for the 111th House of Representatives, at page 0...
Getting bill numbers for the 111th House of Representatives, at page 1...
Getting bill numbers for the 111th House of Representatives, at page 2...
Getting bill numbers for the 111th House of Representatives, at page 3...
Getting bill numbers for the 111th House of Representatives, at page 4...
Getting bill numbers for the 111th House of Representatives, at page 5...
Getting bill numbers for the 111th House of Representatives, at page 6...
Getting bill numbers for the 111th House of Representatives, at page 7...
Getting bill numbers for the 111th House of Representatives, at page 8...
Getting bill numbers for the 111th House of Representatives, at page 9...
Getting bill numbers for the 111th House of Representatives, at page 10...
Getting bill numbers for the 111th House of Representatives, at page 11...
Getting bill numbers for the 111th House of Representatives, at page 12...
Getting bill numbers for the 111th 

I now wish to check how many bill numbers I retrieved, to evaluate whether an Automatic Content Analysis approach is justified within this research.

In [4]:
print("House of Representatives:")

total_number = 0 # I define a counter to get the total number of bills I will analyse.

# I loop over all US Congresses (keys) and unique bill numbers (values) in the "bill_numbers_house" dictionary.
for congress, bill_numbers in bill_numbers_house.items():
    
    # At each iteration, I print the US Congress number (key) and the length of the list of values -
    # i.e., the amount of bill numbers.
    print(f"{congress}th House of Representatives: {len(bill_numbers)} bill numbers")
    
    # At each iteration, I increase the total number of bills by the given amount of bill numbers.
    total_number += len(bill_numbers)

# I print the total number of bills that were introduced in the House of Representatives.
house_number = total_number
print(f"\nThe total number of bills that were introduced in the House of Representatives is {house_number}.")

print("\nSenate:")

# I loop over all US Congresses (keys) and unique bill numbers (values) in the "bill_numbers_senate" dictionary.
for congress, bill_numbers in bill_numbers_senate.items():
    
    # At each iteration, I print the US Congress number (key) and the length of the list of values -
    # i.e., the amount of bill numbers.
    print(f"{congress}th Senate: {len(bill_numbers)} bill numbers")
    
    # At each iteration, I increase the total number of bills by the given amount of bill numbers.
    total_number += len(bill_numbers)
    
# I print the total number of bills that were introduced in the Senate.
senate_number = total_number - house_number
print(f"\nThe total number of bills that were introduced in the Senate is {senate_number}.")
print(f"\nThe total number of bills I wish to retrieve data on is {total_number}.")

House of Representatives:
111th House of Representatives: 6562 bill numbers
115th House of Representatives: 7394 bill numbers

The total number of bills that were introduced in the House of Representatives is 13956.

Senate:
111th Senate: 4059 bill numbers
115th Senate: 3805 bill numbers

The total number of bills that were introduced in the Senate is 7864.

The total number of bills I wish to retrieve data on is 21820.


It seems that, since I wish to retrieve data on 21820 bills, an Automated Content Analysis, and in particular a Supervised Machine Learning approach is justified, since I need to solve two relatively complex textual classification tasks for a high number of documents. I now proceed to save the two dictionaries as .JSON files, so that I do not need to make any additional API request whenever I close the session. I appropriately indent the .JSON files to foster their visual interpretation.

In [5]:
# House of Representatives
with open("bill_numbers_house.json", "w") as w:
    json.dump(bill_numbers_house, w, indent = 4)

# Senate
with open("bill_numbers_senate.json", "w") as w:
    json.dump(bill_numbers_senate, w, indent = 4)

## 3. Getting all the Bill Summaries

Since the data gathering process takes a lot of time, I gather the Summary Data in two separate scripts, one for the House, and one for the Senate, which I will let run on Azure ML. This will allow me to optimise my working schedule, and help me avoiding connection issues. As I have saved the bill numbers for the House of Representatives and the Senate in the `bill_numbers_house.json` and `bill_numbers_senate.json` files, I will be able to make my requests without getting the unique bill identifiers again.

## 4. Getting all the Bill Metadata

Since the data gathering process takes a lot of time, I gather the Bill Metadata in a separate script, which I will let run during the Supervised Machine Learning steps. I only need them for the Data Analysis steps anyway, so this will allow me to optimise my working schedule. As I have saved the bill numbers for the House of Representatives and the Senate in the `bill_numbers_house.json` and `bill_numbers_senate.json` files, I will be able to make my requests without getting the unique bill identifiers again.

## 5. Wrapping Up

On a final note, one may note that the `/summaries/{congress}/{billType}` API endpoint could have saved a lot of time, since I could have paginated over the .JSON responses, getting multiple bills per request, instead of fetching one bill per request. However, the endpoint does not work, and this forces me to go for the long and hard way.